Scenarios are combinations of 
- 2TUL, 3TUL
- No TLO down, CC down, CB down, SO down, EW down
- High stocks, Med stocks, Low stocks
- High rake availability, Low rake availability
- Outload circuit running, Outload circuit down


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as dt

import plotly.express as px

from utilities import run_query

# Create data

In [2]:
def fill(df: pd.DataFrame, status) -> pd.DataFrame:
        df1 = pd.DataFrame(
                {'start': df['end'], 
                'end': df['start'].shift(-1),
                'status': status}
        )
        df1['duration'] = df1.end - df1.start

        return pd.concat([df, df1[:-1]]).sort_values('start').reset_index(drop=True)


In [3]:
df_tul = pd.DataFrame(
    {
        'start': [pd.to_datetime('2024-03-28 08:00:00'), pd.to_datetime('2024-03-29 12:00:00'), pd.to_datetime('2024-03-30 15:30:00'), pd.to_datetime('2024-04-1 15:30:00')],
        'duration': [pd.Timedelta(hours=3), pd.Timedelta(hours=6), pd.Timedelta(hours=1), pd.Timedelta(hours=3)]
    }
)
df_tul['end'] = df_tul.start+df_tul.duration
df_tul['status'] = "2 TUL"

df_tul = fill(df_tul, "3 TUL")
df_tul['name'] = "TUL"

In [4]:
df_tlo = pd.DataFrame(
    {
        'start': [pd.to_datetime('2024-03-28 012:00:00'), pd.to_datetime('2024-03-29 6:00:00'), pd.to_datetime('2024-04-1 15:30:00')],
        'duration': [pd.Timedelta(hours=4), pd.Timedelta(hours=6), pd.Timedelta(hours=5)],
        'status': ["CB Down", 'CB Down', 'EW Down']
    }
)
df_tlo['end'] = df_tlo.start+df_tlo.duration


df_tlo = fill(df_tlo, "OK")
df_tlo['name'] = "TLO"

In [5]:
df_port = pd.DataFrame(
    {
        'start': [pd.to_datetime('2024-03-28 1:00:00'), pd.to_datetime('2024-03-29 18:00:00'), pd.to_datetime('2024-03-30 23:30:00')],
        'end':   [pd.to_datetime('2024-03-29 18:00:00'), pd.to_datetime('2024-03-30 23:30:00'), pd.to_datetime('2024-04-01 19:00:00')],
        'status': ["LOW", 'Med', 'High']
    }
)

df_port['duration'] = df_port.end-df_port.start
df_port['name'] = "Port Stock"

In [6]:
df=pd.concat([df_port, df_tlo, df_tul ])
df1 = df.pivot(index='start', columns='name', values='status').ffill()
df1 = df1.reset_index()
df1['end'] = df1.start.shift(-1)
df1['status'] = df1[[ 'TUL', 'TLO', 'Port Stock']].fillna('').apply(lambda x: ", ".join(x), axis=1)
df1['name'] = "sceanrio"
df1 = df1.dropna()
df1 

name,start,Port Stock,TLO,TUL,end,status,name
3,2024-03-28 12:00:00,LOW,CB Down,3 TUL,2024-03-28 16:00:00,"3 TUL, CB Down, LOW",sceanrio
4,2024-03-28 16:00:00,LOW,OK,3 TUL,2024-03-29 06:00:00,"3 TUL, OK, LOW",sceanrio
5,2024-03-29 06:00:00,LOW,CB Down,3 TUL,2024-03-29 12:00:00,"3 TUL, CB Down, LOW",sceanrio
6,2024-03-29 12:00:00,LOW,OK,2 TUL,2024-03-29 18:00:00,"2 TUL, OK, LOW",sceanrio
7,2024-03-29 18:00:00,Med,OK,3 TUL,2024-03-30 15:30:00,"3 TUL, OK, Med",sceanrio
8,2024-03-30 15:30:00,Med,OK,2 TUL,2024-03-30 16:30:00,"2 TUL, OK, Med",sceanrio
9,2024-03-30 16:30:00,Med,OK,3 TUL,2024-03-30 23:30:00,"3 TUL, OK, Med",sceanrio
10,2024-03-30 23:30:00,High,OK,3 TUL,2024-04-01 15:30:00,"3 TUL, OK, High",sceanrio


In [ ]:

px.timeline(pd.concat([df, df1]), x_start="start", x_end="end", y="name", text='status')

# From Francise


### Rail Delay inside TLO or TUL
a lot of caveats!!!


In [ ]:
run_query('''
    select distinct  "Cause", "Reason"
    from EDW.SELFSERVICE."APLUS_vwEventAllocation"
    where "EventProductionDay" >= '2023-01-01 06:00'
        and (
        ("Circuit" in ('Inload 1', 'Inload 2', 'Inload 3') and "Area" = 'AP Inloading')
        or "Circuit" in ('SM TLO', 'CB TLO', 'CC TLO', 'EW TLO')
        )
        and "Department" = 'Rail'
''')

In [9]:
with pd.option_context('display.max_rows', 50, 'display.max_columns', None): 
  display(
    run_query('''
    select *
    from EDW.SELFSERVICE."APLUS_vwEventAllocation"

    where "EventProductionDay" >= '2023-01-01 06:00'
        and (
        ("Circuit" in ('Inload 1', 'Inload 2', 'Inload 3') and "Area" = 'AP Inloading')
        or "Circuit" in ('SM TLO', 'CB TLO', 'CC TLO', 'EW TLO')
        )
        and "Department" = 'Rail'

    LIMIT 500
        
  ''')
)

,Id,StartTime,EndTime,DurationSeconds,IsManual,EventType,Region,AreaId,Area,SourceEntityName,Circuit,MeasurementPointGroup,MeasurementPoint,EquipmentType,EquipmentId,EquipmentName,Department,Problem,Action,Cause,Reason,TimeUsage,TimeUsageCode,TonnesLoss,EffectiveDuration,EventStatus,Level,DowntimePercent,DowntimeValue,IsParent,LastUpdateTime,ShiftType,EventShift,EventProductionDay,EventDate,EventMonth,EventYear,EventFinYear,EventFinQtr,ParentEventId,Route,MasterEventId
0,2109102,2023-05-08 07:28:20,2023-05-08 07:31:35,195,False,Availability,Eliwana,24,EW Stockyard,EW1.SY.Train Load Out,EW TLO,EW1.Stockyard,EW1.SY.Train Load Out,FMG Train Loadout,27070,TLO901 Train Load Out,Rail,Train speed fluctuation,Train stopped and wagon adjusted,Overloaded Wagons,Incorrect Train Speed,Operating Delay (OD),OD,557.916840,195.0,Allocated,Production,100.0,195,False,2023-05-08 09:20:35.387,Day,2023-05-08 06:00:00,2023-05-08 06:00:00,2023-05-08,2023-May,2023,FY2023,Q4,NaN,Through Loading and Reclaiming,NaN
1,2114867,2023-05-13 15:09:20,2023-05-13 15:33:00,1420,False,Availability,Anderson Point,2,AP Inloading,AP.Inloading 3,Inload 3,AP.Inloading 3,AP.IC3.Inloading Circuit 3,FMG Train Unloader,14391,InCircuit3,Rail,"Completed train 1945. Spot train 1944, CC4, ra...",Tipping train 1944.,Availability Upstream,Spot Train,Operational Standby (SD),SD,25.847222,1420.0,Allocated,Production,100.0,1420,False,2023-05-13 16:59:26.320,Day,2023-05-13 06:00:00,2023-05-13 06:00:00,2023-05-13,2023-May,2023,FY2023,Q4,NaN,None,NaN
2,2664835,2024-02-18 09:29:06,2024-02-18 09:44:00,894,False,Availability,Anderson Point,2,AP Inloading,AP.Inloading 2,Inload 2,AP.Inloading 2,AP.IC2.Inloading Circuit 2,FMG Train Unloader,14324,InCircuit2,Rail,Train 0591 spotted.,Train 0591 commenced dumping to D3.,Availability Upstream,Waiting For Next Train,Operational Standby (SD),SD,0.000000,890.0,Allocated,Production,100.0,894,False,2024-02-18 09:56:27.053,Day,2024-02-18 06:00:00,2024-02-18 06:00:00,2024-02-18,2024-Feb,2024,FY2024,Q3,2664761.0,None,2664761.0
3,2193003,2023-07-13 10:42:10,2023-07-13 11:12:30,1820,False,Availability,Anderson Point,2,AP Inloading,AP.Inloading 3,Inload 3,AP.Inloading 3,AP.IC3.Inloading Circuit 3,FMG Train Unloader,14391,InCircuit3,Rail,"Train 2901 tipping completed, Train 2900 at LOA",Train 2900 spotted into cells and tipping comm...,Availability Upstream,Spot Train,Operational Standby (SD),SD,16.420833,1820.0,Allocated,Production,100.0,1820,False,2023-07-13 11:15:03.933,Day,2023-07-13 06:00:00,2023-07-13 06:00:00,2023-07-13,2023-Jul,2023,FY2024,Q1,NaN,None,NaN
4,2383166,2023-10-22 01:01:50,2023-10-22 01:22:40,1250,False,Availability,Christmas Creek,8,CC Stockyard,CC Stockyard,CC TLO,CCS.Train Loadout,CCS.TLO.Train Loadout Circuit,FMG Train Loadout,18611,CC Outloading Circuit,Rail,"Finished train, next empty at LOA",Waiting to clear train.,Availability Downstream,Waiting To Clear Train,Operational Standby (SD),SD,0.000000,1250.0,Allocated,Production,100.0,1250,False,2023-10-22 01:27:13.687,Night,2023-10-21 18:00:00,2023-10-21 06:00:00,2023-10-21,2023-Oct,2023,FY2024,Q2,NaN,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2210533,2023-07-26 08:46:10,2023-07-26 08:48:15,125,False,Availability,Solomon,17,SM Stockyard,SM Stockyard,SM TLO,SM.Train Loadout,SM.TLO.Train Loadout,FMG Train Loadout,23314,SM TLO Circuit,Rail,Driver change,Driver change,Availability Downstream,Waiting For Train – Driver Change,Operational Standby (SD),SD,8.197223,125.0,Allocated,Production,100.0,125,False,2023-07-26 09:05:11.190,Day,2023-07-26 06:00:00,2023-07-26 06:00:00,2023-07-26,2023-Jul,2023,FY2024,Q1,NaN,None,NaN
496,2017094,2023-02-18 10:43:00,2023-02-18 10:59:00,960,False,Availability,Solomon,17,SM Stockyard,SM Stockyard,SM TLO,SM.Train Loadout,SM.TLO.Train Loadout,FMG Train Loadout,23314,SM TLO Circuit,Rail,Wait for train to depart,Wait fo

In [3]:
df = run_query('''
    select  "Problem"
    from EDW.SELFSERVICE."APLUS_vwEventAllocation"
    where "EventProductionDay" >= '2023-01-01 06:00'
        and "Circuit" in ('SM TLO', 'CB TLO', 'CC TLO', 'EW TLO')
        and "Department" = 'Rail'
''')
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(df)

,Problem
0,Wait for next train
1,waiting for train
2,driver change
3,No train scheduled
4,Wagon overloaded
5,Overloaded wagons.
6,No train scheduled- slow trains after track re...
7,Wagon adjustment Caused by incorrect train spe...
8,Waiting for Train due to breakdown
9,Overloaded wagon on restart from driver change...


### Port and TLO APLUS delays

I have Area = AP Inloading and Circuit in (IL1, IL2, IL3) clause because Inload 4, the CHF / magnetite circuit is also under Area = AP Inloading In addition, the circuit is not 100% locked to their area - there are some Inload circuit delays incorrectly allocated to Area = AP Outloading

In [3]:
run_query("""select *
from EDW.SELFSERVICE."APLUS_vwEventAllocation"
where "EventProductionDay" = '2024-03-21 06:00'
    and (
    ("Circuit" in ('Inload 1', 'Inload 2', 'Inload 3') and "Area" = 'AP Inloading')
    or ("Circuit" in ('Outload 1', 'Outload 2', 'Outload 3') and "Area" = 'AP Outloading')
    or "Circuit" in ('SM TLO', 'CB TLO', 'CC TLO', 'EW TLO')
    )""")

,Id,StartTime,EndTime,DurationSeconds,IsManual,EventType,Region,AreaId,Area,SourceEntityName,...,EventShift,EventProductionDay,EventDate,EventMonth,EventYear,EventFinYear,EventFinQtr,ParentEventId,Route,MasterEventId
0,2745931,2024-03-22 04:44:00,2024-03-22 04:48:00,240,True,Availability,Christmas Creek,8,CC Stockyard,None,...,2024-03-21 18:00:00,2024-03-21 06:00:00,2024-03-21,2024-Mar,2024,FY2024,Q3,NaN,None,NaN
1,2744664,2024-03-22 04:23:41,2024-03-22 04:26:14,153,False,Availability,Anderson Point,3,AP Outloading,AP.Outloading 2,...,2024-03-21 18:00:00,2024-03-21 06:00:00,2024-03-21,2024-Mar,2024,FY2024,Q3,2744658.0,None,2744283.0
2,2744669,2024-03-22 01:24:07,2024-03-22 01:26:38,151,False,Availability,Anderson Point,3,AP Outloading,AP.Outloading 3,...,2024-03-21 18:00:00,2024-03-21 06:00:00,2024-03-21,2024-Mar,2024,FY2024,Q3,2744236.0,None,2744236.0
3,2744650,2024-03-22 01:23:51,2024-03-22 01:30:47,416,False,Availability,Anderson Point,3,AP Outloading,AP.Outloading 2,...,2024-03-21 18:00:00,2024-03-21 06:00:00,2024-03-21,2024-Mar,2024,FY2024,Q3,2744283.0,None,2744283.0
4,2744133,2024-03-21 21:32:10,2024-03-21 21:34:15,125,False,Loss,Anderson Point,3,AP Outloading,AP.Outloading 3,...,2024-03-21 18:00:00,2024-03-21 06:00:00,2024-03-21,2024-Mar,2024,FY2024,Q3,NaN,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,2742755,2024-03-21 07:28:10,2024-03-21 07:32:01,231,False,Availability,Solomon,17,SM Stockyard,SM Stockyard,...,2024-03-21 06:00:00,2024-03-21 06:00:00,2024-03-21,2024-Mar,2024,FY2024,Q3,NaN,None,NaN
271,2742853,2024-03-21 08:48:40,2024-03-21 08:50:41,121,False,Availability,Solomon,17,SM Stockyard,SM Stockyard,...,2024-03-21 06:00:00,2024-03-21 06:00:00,2024-03-21,2024-Mar,2024,FY2024,Q3,NaN,None,NaN
272,2743256,2024-03-21 13:15:02,2024-03-21 15:05:01,6599,False,Availability,Solomon,17,SM Stockyard,SM Stockyard,...,2024-03-21 06:00:00,2024-03-21 06:00:00,2024-03-21,2024-Mar,2024,FY2024,Q3,2743171.0,None,2743171.0
273,2744661,2024-03-22 02:39:59,2024-03-22 03:35:44,3345,False,Availability,Anderson Point,3,AP Outloading,AP.Outloading 2,...,2024-03-21 18:00:00,2024-03-21 06:00:00,2024-03-21,2024-Mar,2024,FY2024,Q3,2744658.0,None,2744283.0



### Port and TLO APLUS delays

select *
from EDW.SELFSERVICE."APLUS_vwEventAllocation"
where "EventProductionDay" = '2024-03-21 06:00'
    and (
    ("Circuit" in ('Inload 1', 'Inload 2', 'Inload 3') and "Area" = 'AP Inloading')
    or ("Circuit" in ('Outload 1', 'Outload 2', 'Outload 3') and "Area" = 'AP Outloading')
    or "Circuit" in ('SM TLO', 'CB TLO', 'CC TLO', 'EW TLO')
    )
-- -- -- I have Area = AP Inloading and Circuit in (IL1, IL2, IL3) clause because Inload 4, the CHF / magnetite circuit is also under Area = AP Inloading
-- -- -- in addition, the circuit is not 100% locked to their area - there are some Inload circuit delays incorrectly allocated to Area = AP Outloading

;


-- -- -- Rail delays logged by Train Control (outside of TLO and TUL)


select *, TRAINID."NumValue" as "TrainID"
from EDW.SELFSERVICE."APLUS_vwEventAllocation" as EV
left join "EDW"."STG_APLUS"."EventUserFieldValue" as TRAINID on EV."Id" = TRAINID."EventId" and TRAINID."UserFieldId" = 2004 and TRAINID."EffectiveToDate" is null   -- TrainID
where "EventProductionDay" = '2024-03-21 06:00' and "Area" = 'Rail Network';


-- -- -- Rail Delay inside TLO or TUL
-- -- -- a lot of caveats!!!
select distinct  "Cause", "Reason"
from EDW.SELFSERVICE."APLUS_vwEventAllocation"
where "EventProductionDay" >= '2019-01-01 06:00'
    and (
    ("Circuit" in ('Inload 1', 'Inload 2', 'Inload 3') and "Area" = 'AP Inloading')
    or "Circuit" in ('SM TLO', 'CB TLO', 'CC TLO', 'EW TLO')
    )
    and "Department" = 'Rail'


- 2TUL, 3TUL – from APLUS, see attached query
- No TLO down, CC down, CB down, SO down, EW down] – from APLUS, see attached query
- High stocks, Med stocks, Low stocks] – I get my stock level at 0600 from AA_OPERATIONS_MANAGEMENT.SELFSERVICE.INVENTORY_STOCKPILE_TRANSACTIONS_35_DAYS_6AM. Suggest Damon Hong or Christ Pavlinovich’s team to help with this.
- High rake availability, Low rake availability] – APLUS, see attached query
- Outload circuit running, Outload circuit down] – from APLUS, see attached query




I’ve this dashboard that plots how many TULs are being ‘occupied’ and how many trains queueing here , idea that I borrowed from James Griggs. I defined occupied as during unloading, or in between train with a queue, or a non-queued train has arrived Port Limit of Authority, or if there is a TUL delay that is not Availability Upstream (waiting for next train type). The data processing is in View Definition of AA_OPERATIONS_HIVEANALYTICS.SELFSERVICE.IOPS_VW_TUL_OCCUPANCY_AND_TRAIN_QUEUE

 

 

 

Limitations of the data

OPF and Port APLUS data is fine for certain things but can be quite inaccurate for some applications.
I use this data for
crude reliability engineering analysis to know what the top unscheduled maintenance (UM) or operating delays (OD) are, to point me to further investigation or work prioritisation
complement other data sources to get the context behind certain delays, or use other data sources to improve the APLUS data quality.
Start and End time are generally quite accurate because they are system generated. Error usually comes from manually entered data.
Big long delays like multi-days belt rip / major equipment failure are accurate.
Limitations /  issues that I’m aware of
Incorrect data, generally Operational Standby (someone else holding me up type delays) are less reliable. E.gs
Interface point - Cloudbreak and Christmas Creek tend to log the time between trains under Waiting for Next Train, even though there is already a train waiting at the Limit of Authority (LOA). Quite often when you read delays logged by Train Control, they will record that time as TLO waiting for Product or equipment issues. For TLO-Rail interface analysis, I supplement that with Trainlog data to see where the trains are, to separate waiting for next train, from say spot / clear train or hold up in getting train into TLO.
No overlapping delay protection, e.g. CC Waiting for Next Train delay (Master Event ID 2190628) with end time incorrectly set to the following month, and spawned a 12 hours delay each shift for 30 days. Some were deleted but over 24 days worth were left in the system for >9 months. During that time other delays were still been logged. Sometimes I do a if EndTime > Next StartTime then set the EndTime to Next StartTime to get around this. If the error is big enough, I’ll also email Control Room to fix.  
Scheduled Maintenance as a bucket is generally okay, just that they sometimes / often log Scheduled Maintenance Overrun as Scheduled Maintenance still. You can compare that to the official shutdown calendar to get the SMO hours.  That said  EW has logged a few days of their shutdown to Waiting for Next Train.
Actual allocation themselves are what they think were the correct cause – reason at the time. E.g. for recurring faults, after extensive troubleshooting they may found the true cause but they may not go back to reallocate all the entries to the true cause.  
Rail APLUS data  
each section has a target duration, and they get prompt to log a delay if exceed that duration. The start and end time is within that section, but not necessarily the actual train stoppage time.
The accounting point is at the ore train. A track fault / work or locomotive fault is only visible in APLUS if the fault delay a train. If you want accurate picture of Track work, use RailEdge and SAP. Locomotive – REMs is available, but I find it not reliable as it relies on RSM supervisors to manually log the delays. Also, a 1hr track fault could affect 0 train (thus no APLUS delay), or all 17 trains.
An ore train can have different consist – they can swap locos and ore cars and still call it Rake 01.
They can decide to run, say, a 120 car rake (Rake 18) for a few days so those few days will have 18 rakes worth of APLUS delays. I’ve processed the number of rake each day into AA_OPERATIONS_HIVEANALYTICS.SELFSERVICE.IOPS_TBL_RAIL_DAILY_RAKE_NUMBER
 


Stock level
CB, although the system may shows they have >500kt stock, they have a lower limit to operate their vault effectively so they will try to avoid going below that level, unless they want to empty it for say a shutdown. Not sure what is the magic number is and how they decide.
Total stock vs reclaimable stock – total stock might be high, but made up of incomplete stockpiles that they can’t reclaim yet. Rob Walsh has some rules that he uses to define reclaimable stockpile at Port.